<a href="https://colab.research.google.com/github/UDICatNCHU/PyTorch-SocialNetwork/blob/master/word2vec_skip_gram_social_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.functional as F
import torch.nn.functional as F

In [0]:
corpus = [
    'he is a king',
    'she is a queen',
    'he is a man',
    'she is a woman',
    'warsaw is poland capital',
    'berlin is germany capital',
    'paris is france capital',   
]

In [0]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

tokenized_corpus = tokenize_corpus(corpus)

In [0]:
vocabulary = []
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

In [9]:
idx2word

{0: 'he',
 1: 'is',
 2: 'a',
 3: 'king',
 4: 'she',
 5: 'queen',
 6: 'man',
 7: 'woman',
 8: 'warsaw',
 9: 'poland',
 10: 'capital',
 11: 'berlin',
 12: 'germany',
 13: 'paris',
 14: 'france'}

In [0]:
window_size = 2
idx_pairs = []
# for each sentence
for sentence in tokenized_corpus:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

In [7]:
idx_pairs

array([[ 0,  1],
       [ 0,  2],
       [ 1,  0],
       [ 1,  2],
       [ 1,  3],
       [ 2,  0],
       [ 2,  1],
       [ 2,  3],
       [ 3,  1],
       [ 3,  2],
       [ 4,  1],
       [ 4,  2],
       [ 1,  4],
       [ 1,  2],
       [ 1,  5],
       [ 2,  4],
       [ 2,  1],
       [ 2,  5],
       [ 5,  1],
       [ 5,  2],
       [ 0,  1],
       [ 0,  2],
       [ 1,  0],
       [ 1,  2],
       [ 1,  6],
       [ 2,  0],
       [ 2,  1],
       [ 2,  6],
       [ 6,  1],
       [ 6,  2],
       [ 4,  1],
       [ 4,  2],
       [ 1,  4],
       [ 1,  2],
       [ 1,  7],
       [ 2,  4],
       [ 2,  1],
       [ 2,  7],
       [ 7,  1],
       [ 7,  2],
       [ 8,  1],
       [ 8,  9],
       [ 1,  8],
       [ 1,  9],
       [ 1, 10],
       [ 9,  8],
       [ 9,  1],
       [ 9, 10],
       [10,  1],
       [10,  9],
       [11,  1],
       [11, 12],
       [ 1, 11],
       [ 1, 12],
       [ 1, 10],
       [12, 11],
       [12,  1],
       [12, 10],
       [10,  1

In [0]:
def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size, dtype=torch.float)
    x[word_idx] = 1.0
    return x

In [43]:
embedding_dims = 5
# W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
W1 = torch.rand(embedding_dims, vocabulary_size, requires_grad=True)
# W1.requires_grad = True
# W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
W2 = torch.rand(vocabulary_size, embedding_dims, requires_grad=True)
# W2.requires_grad = True
num_epochs = 1001
learning_rate = 0.001

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:
        x = get_input_layer(data)
        y_true = torch.tensor(np.array([target]), dtype=torch.long)
        z1 = torch.matmul(W1, x)
        z2 = torch.matmul(W2, z1)
        loss = F.cross_entropy(z2.view(1,-1), y_true)
        loss_val += loss.item()
        loss.backward()
        W1.data -= learning_rate * W1.grad
        W2.data -= learning_rate * W2.grad
        W1.grad.data.zero_()
        W2.grad.data.zero_()
    if epo % 10 == 0:    
        print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')

Loss at epo 0: 2.741964670589992
Loss at epo 10: 2.680916748728071
Loss at epo 20: 2.626224524634225
Loss at epo 30: 2.577154949733189
Loss at epo 40: 2.533135373251779
Loss at epo 50: 2.493671294621059
Loss at epo 60: 2.4582931143896922
Loss at epo 70: 2.426531822340829
Loss at epo 80: 2.3979229296956746
Loss at epo 90: 2.3720299397196087
Loss at epo 100: 2.3484697716576712
Loss at epo 110: 2.3269288676125663
Loss at epo 120: 2.3071653519357955
Loss at epo 130: 2.2889973878860475
Loss at epo 140: 2.272287210396358
Loss at epo 150: 2.2569222586495536
Loss at epo 160: 2.2428015266145978
Loss at epo 170: 2.229827632222857
Loss at epo 180: 2.2179020404815675
Loss at epo 190: 2.206925548825945
Loss at epo 200: 2.19679993220738
Loss at epo 210: 2.1874301842280794
Loss at epo 220: 2.1787268570491247
Loss at epo 230: 2.1706077780042374
Loss at epo 240: 2.1629985468728203
Loss at epo 250: 2.1558328356061662
Loss at epo 260: 2.1490527050835744
Loss at epo 270: 2.142607113293239
Loss at epo 280: